# Spark Preparation
We check if we are in Google Colab.  If this is the case, install all necessary packages.

To run spark in Colab, we need to first install all the dependencies in Colab environment i.e. Apache Spark 3.3.2 with hadoop 3.3, Java 8 and Findspark to locate the spark in the system. The tools installation can be carried out inside the Jupyter Notebook of the Colab.
Learn more from [A Must-Read Guide on How to Work with PySpark on Google Colab for Data Scientists!](https://www.analyticsvidhya.com/blog/2020/11/a-must-read-guide-on-how-to-work-with-pyspark-on-google-colab-for-data-scientists/)

In [1]:
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False

In [2]:
if IN_COLAB:
    !apt-get install openjdk-8-jdk-headless -qq > /dev/null
    !wget -q https://dlcdn.apache.org/spark/spark-3.3.2/spark-3.3.2-bin-hadoop3.tgz
    !tar xf spark-3.3.2-bin-hadoop3.tgz
    !mv spark-3.3.2-bin-hadoop3 spark
    !pip install -q findspark
    import os
    os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
    os.environ["SPARK_HOME"] = "/content/spark"

# Start a Local Cluster

In [3]:
import findspark
findspark.init()

In [ ]:
spark_url = 'local'

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext

spark = SparkSession.builder\
        .master(spark_url)\
        .appName('Spark Tutorial')\
        .config('spark.ui.port', '4040')\
        .getOrCreate()

In [ ]:
from pyspark.sql.functions import avg, min, max, countDistinct


# Spark Assignment

Based on the movie review dataset in 'netflix-rotten-tomatoes-metacritic-imdb.csv', answer the below questions.

**Note:** do not clean or remove missing data

In [ ]:
path = '/content/netflix-rotten-tomatoes-metacritic-imdb.csv'

In [ ]:
df = spark.read.option("header", True).csv(path)


In [ ]:
df.show()

+--------------------+--------------------+--------------------+--------------------+---------------+----------------+--------------------+------------+--------------------+--------------------+--------------------+-----------+----------+---------------------+----------------+---------------+--------------------+------------+------------+--------------------+--------------------+--------------------+--------------------+--------------------+----------+--------------------+--------------------+--------------------+------------+
|               Title|               Genre|                Tags|           Languages|Series or Movie|Hidden Gem Score|Country Availability|     Runtime|            Director|              Writer|              Actors|View Rating|IMDb Score|Rotten Tomatoes Score|Metacritic Score|Awards Received|Awards Nominated For|   Boxoffice|Release Date|Netflix Release Date|    Production House|        Netflix Link|           IMDb Link|             Summary|IMDb Votes|           

In [ ]:
df.columns

['Title',
 'Genre',
 'Tags',
 'Languages',
 'Series or Movie',
 'Hidden Gem Score',
 'Country Availability',
 'Runtime',
 'Director',
 'Writer',
 'Actors',
 'View Rating',
 'IMDb Score',
 'Rotten Tomatoes Score',
 'Metacritic Score',
 'Awards Received',
 'Awards Nominated For',
 'Boxoffice',
 'Release Date',
 'Netflix Release Date',
 'Production House',
 'Netflix Link',
 'IMDb Link',
 'Summary',
 'IMDb Votes',
 'Image',
 'Poster',
 'TMDb Trailer',
 'Trailer Site']

## What is the maximum and average of the overall hidden gem score?

In [ ]:
df = df.withColumn('Hidden Gem Score', df['Hidden Gem Score'].cast('float'))
df.select(avg('Hidden Gem Score'), min('Hidden Gem Score'), max('Hidden Gem Score')).show()


+---------------------+---------------------+---------------------+
|avg(Hidden Gem Score)|min(Hidden Gem Score)|max(Hidden Gem Score)|
+---------------------+---------------------+---------------------+
|    5.937551392466598|                  0.6|                  9.8|
+---------------------+---------------------+---------------------+



## How many movies that are available in Korea?

In [ ]:
from pyspark.sql.functions import array_contains
from pyspark.sql.functions import split
from pyspark.sql.functions import split, explode

In [ ]:
df2 = df.withColumn("Language", split("Languages", ",\s*"))

In [ ]:
df2 = df2.filter(array_contains("Language", "Korean"))

In [ ]:
df2.count()

735

## Which director has the highest average hidden gem score?

In [ ]:
from pyspark.sql.functions import col, desc

In [ ]:
df3 = df.withColumn('Hidden Gem Score', df['Hidden Gem Score'].cast('float'))
df3 = df3.groupby('Director').agg(avg("Hidden Gem Score").alias("Avg Hidden Gem Score"))
df3.sort(col("Avg Hidden Gem Score").desc()).show(1)

+-----------+--------------------+
|   Director|Avg Hidden Gem Score|
+-----------+--------------------+
|Dorin Marcu|   9.800000190734863|
+-----------+--------------------+
only showing top 1 row



## How many genres are there in the dataset?

In [ ]:
df4 = df.filter(df['Genre'] != 'null')
df5 = df4.select(split(col("Genre"), ",\s*").alias("Genres_array"))
df5 = df5.select(explode(col("Genres_array")).alias("Genres"))
df6 = df5.groupby(df5["Genres"]).count()


In [ ]:
df6.count()

28